## Linear Regression as a Machine Learning Model
In this notebook, we will revisit linear regression and consider implementing it as a machine learning model.

### Import modules
Begin by importing the modules to be used in this notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Linear Regression Revisited
In previous lectures, we used regression to fit linear curves to our data.

For example, consider the following dataset:

In [ ]:
data = np.genfromtxt('linear_scatter.csv',delimiter=',')
x = data[:,0]
y = data[:,1]

Perhaps this data pertains to kelp canopy height depending on weeks in the season:

In [ ]:
plt.plot(x,y,'k.')
plt.xlabel('x (time since June 1, weeks)')
plt.ylabel('y (canopy height, m)')
plt.show()

We can fit a polynomial to this data using numpy as follows:

In [ ]:
p = np.polyfit(x,y,deg=1)
slope_fit = p[0]
intercept_fit = p[1]
print(intercept_fit, slope_fit)

This slope and intercept defines a growth rate through the season, We can plot the model on an independent x-axis as follows:

In [ ]:
ordered_x = np.arange(0,10,0.1)
model_y = np.poly1d(p)
fit_y = model_y(ordered_x)

plt.plot(x,y,'k.',label='data')
plt.plot(ordered_x, fit_y,'b-',label='model')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc=2)
plt.show()

In this process, we minimized the error between the points and the line to find the best estimates of the slope $m$ and the slope $b$. We could write this as a cost function as
$$
 J = \sqrt{\frac{1}{N} \sum_{i=1}^N (y_{data}(i) - y_{model}(i))^2}
$$
and code it up in numpy as:

In [ ]:
def cost_function(y_data, y_modeled):
    # RMSE
    N = np.size(y_data)
    squared_error = (1/N)*np.sum((y_data-y_modeled)**2)
    return(np.sqrt(squared_error))

Using the above cose function, we can compute the "error space" -- the cost between data and model for a range of the parameters $m$ and $b$.

In [ ]:
intercept_space = np.linspace(-20,50,100)
slope_space = np.linspace(1,12,100)
I, S = np.meshgrid(intercept_space, slope_space)
Error = np.zeros((100,100))

# fill in the error matrix
for row in range(np.shape(I)[0]):
    for col in range(np.shape(S)[1]):
        Error[row,col] = cost_function(y, I[row,col]+S[row,col]*x)

Then, we can visualize our best estimate on this error space:

In [ ]:
C = plt.pcolormesh(intercept_space,slope_space, np.log10(Error))
plt.contour(intercept_space,slope_space, np.log10(Error),colors='white',linewidths=0.7)
plt.plot(intercept_fit, slope_fit, 'wo')
plt.text(intercept_fit+2, slope_fit, '$\leftarrow$ Best Fit Parameters',color='white',va='center')
plt.colorbar(C, label='log(cost)')
plt.xlabel('intercept ($b$)')
plt.ylabel('slope ($m$)')
plt.show()

As we can see, out best fit parameters are those which minimize the error.

We could also approach this as an iterative process instead of a deterministic process. In other words, if we have a guess at a set of parameters, we can use the geometry of the error space to determine the right set of parameters by making updates to our guess. To facilitate this process, we need to compute the gradient of our cost function.

After we've computed the cost, we code it up in numpy:

In [ ]:
### code up the cost_function_gradient function here


To use our cost function graident, let's make an initial guess and define a learning rate:

In [ ]:
learning_rate = 0.02
intercept = 10.0 # starting intercept guess
slope = 2.0 # starting slope guess
weights = np.array([intercept, slope])

After one iteration, we can update our initial guess as follows:

In [ ]:
# update the weights using gradient descent


We can imagine doing this over and over again until we converge on a best estimate. Let's build a slider to examine how this would look over many iterations

In [ ]:
def plot_fit_and_cost(initial_guess, n_iterations):

    # initial_guess = np.array([10.0, 2.0])
    # x_data = x
    # y_data = y
    # n_iterations = 1

    weights = np.copy(initial_guess)
    for n in range(n_iterations):
        y_modeled = weights[0]+weights[1]*x
        weight_gradient = cost_function_gradient(x, y, y_modeled)
        weights -= learning_rate*weight_gradient
    
    fig = plt.figure(figsize=(11,5))
    
    plt.subplot(1,2,1)
    plt.plot(x, y,'k.')
    plot_x = np.linspace(np.min(x), np.max(x), 100)
    modeled_y = weights[0] + weights[1]*plot_x
    plt.plot(plot_x, modeled_y,'b-',
            label='Fit: '+'{:.2f}'.format(weights[1])+'*x + ' +'{:.2f}'.format(weights[0]))
    plt.plot(plot_x, intercept_fit + slope_fit*plot_x,'b--',
             label='Best Fit: '+'{:.2f}'.format(slope_fit)+'*x + ' +'{:.2f}'.format(intercept_fit))
    plt.title('Fit after '+str(n_iterations)+' iteration(s)')
    plt.legend(loc=2)
    plt.ylabel('y')
    plt.xlabel('x')
    
    plt.subplot(1,2,2)
    C = plt.pcolormesh(intercept_space,slope_space, np.log10(Error))
    plt.contour(intercept_space,slope_space, np.log10(Error),colors='white',linewidths=0.7)
    plt.plot(initial_guess[0], initial_guess[1], 'wo')
    plt.plot(weights[0], weights[1], 'wo')
    plt.text(initial_guess[0]+2, initial_guess[1], '$\leftarrow$ Initial',color='white',va='center')
    if n_iterations>0:
        plt.text(weights[0]+2, weights[1], '$\leftarrow$ Final',color='white',va='center')
    plt.colorbar(C, label='log(cost)')
    plt.title('Error space')
    plt.ylabel('slope ($m$)')
    plt.xlabel('intercept ($b$)')
    plt.show()


In [ ]:
interact(plot_fit_and_cost, initial_guess=fixed(np.array([intercept, slope])),
         n_iterations=widgets.IntSlider(min=0, max=200));

In the above demonstration, we have "learned" the right parameters for our model by "training" it on our data